In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import re
import torch.nn as nn
import time

from sklearn.model_selection import GroupKFold

import albumentations 
from albumentations.pytorch.transforms import ToTensorV2

import torch
import torchvision
from albumentations import transforms
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN

from torch.utils.data import DataLoader, Dataset

from matplotlib import pyplot as plt

In [ ]:
train_df = pd.read_csv("../input/vinbigdata-256-image-dataset/vinbigdata/train.csv")
sample = pd.read_csv("../input/vinbigdata-chest-xray-abnormalities-detection/sample_submission.csv")

IMG_SIZE = 256
train_df['xmin'] = (train_df['x_min']/train_df['width'])*IMG_SIZE
train_df['ymin'] = (train_df['y_min']/train_df['height'])*IMG_SIZE
train_df['xmax'] = (train_df['x_max']/train_df['width'])*IMG_SIZE
train_df['ymax'] = (train_df['y_max']/train_df['height'])*IMG_SIZE

In [ ]:
df = train_df

In [ ]:
sample.head()

In [ ]:
def make_folds(df):
    
    df["kfold"] = -1
    df = df.sample(frac=1).reset_index(drop=True)
    y = df.class_id.values
    
    #Group K Fold
    gf = GroupKFold(n_splits=5)
    
    #We group by image_id because the same group should not appear in two different folds
    for idx, (train_idx, valid_idx) in enumerate(gf.split(X=df, y=y, groups=df.image_id.values)):
        df.loc[valid_idx, "kfold"] = idx

    return df

df = make_folds(df)
print(df.head())

In [ ]:
df = df[df['class_id'] != 14].reset_index(drop=True)

In [ ]:
#df.fillna(0, inplace=True)
#df.loc[df["class_id"] == 14, ['x_max', 'y_max']] = 1.0


# FasterRCNN handles class_id==0 as the background.
# df["class_id"] = df["class_id"] + 1
# df.loc[df["class_id"] == 15, ["class_id"]] = 0

# df.shape

In [ ]:
df["class_name"].nunique()

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.sort_values(by="class_id").head(20)

In [ ]:
classes = df["class_name"].unique().tolist()
total_classes = ["Unknown"] + classes

In [ ]:
df["image_id"].shape

In [ ]:
df.head()

In [ ]:
config = {
    "image_dir":"../input/vinbigdata-256-image-dataset/vinbigdata/train",
    "num_epochs": 10,
    "lr_rate": 1e-6,
    "num_classes":15,
    "batch_size":8,
    "test_dir": "../input/vinbigdata-256-image-dataset/vinbigdata/test"
}

In [ ]:
class VinBigData(Dataset):
    def __init__(self, df, img_dir, istransforms=None, transforms=None, isTest=False):
        super(VinBigData, self).__init__()
        self.df = df
        self.image_ids = self.df["image_id"].unique()
        self.img_dir = img_dir
        self.istransforms = istransforms
        self.transforms = transforms
        self.isTest = isTest
    
    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        
        record = self.df[self.df["image_id"] == image_id]
        image = cv2.imread(f'{self.img_dir}/{image_id}.png', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        
        if self.isTest == True:
            image = np.transpose(image, (2, 1, 0))
            return torch.tensor(image, dtype=torch.float32), image_id

        bb_box = record[["xmin","ymin","xmax","ymax"]].values
        area = (bb_box[:, 3] - bb_box[:,1]) * (bb_box[:,2] - bb_box[:, 0])
        
        iscrowd = torch.ones((record.shape[0],), dtype=torch.float32)
        
        area = torch.tensor(area, dtype=torch.float32)
        
        labels = torch.as_tensor((record.class_id.values + 1), dtype=torch.int64)
        
        targets = {}
        targets["labels"] = labels
        targets["area"] = area
        targets["iscrowd"] = iscrowd
        targets["image_id"] = torch.tensor([idx])
        targets["boxes"] = torch.tensor(bb_box, dtype=torch.float32)
        
        if self.istransforms is not None:
            image = self.transforms(image)
        

        return image, targets


In [ ]:
data = VinBigData(df, img_dir=config["image_dir"])

In [ ]:
data[0][1]

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

num_classes = 15

in_features = model.roi_heads.box_predictor.cls_score.in_features

model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [ ]:
model

In [ ]:
from torchvision import transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
        transforms.append(T.Normalize(mean=(0,0,0), std=(1,1,1)))
    else:
        transforms.append(T.Normalize(mean=(0,0,0), std=(1,1,1)))
    return T.Compose(transforms)



In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

dataset = VinBigData(df, config["image_dir"], istransforms=True, transforms=get_transform(train=True))
data_loader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True, num_workers=4, collate_fn=collate_fn)

images, targets = next(iter(data_loader))
images = list(image  for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images, targets)

model.eval()
x = [torch.randn(3, 256,256), torch.randn(3, 500, 400)]
predictions= model(x)
print(predictions)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Device: {device}")
    
model.to(device)

lr_rate = 1e-6

epochs = 20

lr_scheduler = None

params = [p for p in model.parameters() if p.requires_grad]

optimizer = torch.optim.Adam(model.parameters(), lr=lr_rate)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)

num_folds = 5

In [ ]:
from tqdm import tqdm

num_epochs = config["num_epochs"]

dataset = VinBigData(df, config["image_dir"], istransforms=True, transforms=get_transform(train=True))
train_data_loader = torch.utils.data.DataLoader(dataset, batch_size=config["batch_size"], shuffle=True, num_workers=4, collate_fn=collate_fn)
itr = 1
num_imgs = 0

for epoch in range(num_epochs):
    
    loss_value = 0.0
    num_images= 0
    loop = tqdm(enumerate(train_data_loader), total = len(train_data_loader))
    
    for batch (images, targets) in tqdm(loop):
        
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        
        print(type(loss_dict))
        losses = sum(loss for loss in loss_dict.values())
        loss_value += losses.item() 
        num_images += len(images)
        
        print("Loss:{} num_images {}".format(loss_value, num_images))
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if itr % 20 == 0:
            print(f"Iteration #{itr} loss: {loss_value/itr}")

        itr += 1
        

    
    # update the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()

    print(f"Epoch #{epoch} loss: {loss_value/len(train_data_loader)}")   
    # print("Saving epoch's state...")
    # torch.save(model.state_dict(), f"model_state_epoch_{epoch}.pth")

In [ ]:
from tqdm import tqdm

def run(validation_index):

    train_data = df
    #train_data = df[df["kfold"] != validation_index]
    test_data = df[df["kfold"] == validation_index]
    
    dataset = VinBigData(train_data, config["image_dir"], istransforms=True, transforms=get_transform(train=True))
    train_data_loader = torch.utils.data.DataLoader(dataset, batch_size=config["batch_size"], shuffle=True, num_workers=4, collate_fn=collate_fn)
    
    dataset = VinBigData(test_data, config["image_dir"], istransforms=True, transforms=get_transform(train=False))
    valid_data_loader = torch.utils.data.DataLoader(dataset, batch_size=config["batch_size"], shuffle=True, num_workers=4, collate_fn=collate_fn) 
    
    
    training_dict = {"train":train_data_loader, "valid":valid_data_loader}
    
    for epoch in range(config["num_epochs"]):
        
        
        
        num_correct = 0
        current_loss = 0.0

        best_loss = float("inf")
        
        for phase in ["train"]:
            
            if phase == "train":
                scheduler.step()
                model.train()

                
            num_samples = 0
            loss_value = 0.0
            
            loop = tqdm(enumerate(training_dict[phase]), total = len(training_dict[phase]))
            
            for batch, (train_images, targets) in loop:
                
                
                train_images = list(image.to(device) for image in train_images)
                
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
                
                loss_dict = model(train_images, targets)
                
                losses = sum(loss for loss in loss_dict.values())
                
                loss_value += losses.item()
                
                optimizer.zero_grad()
                num_samples += 1
                
                with torch.set_grad_enabled(phase == "train"):
                    
                    losses.backward()
                    optimizer.step()
            
            loss_value = loss_value / len(training_dict[phase])
            print('{}/{} {} Loss: {:.4f}'.format(epoch, config["num_epochs"], phase, loss_value))
            
            avg_loss = loss_value
            if avg_loss < best_loss:
                best_loss = avg_loss
                print('Best loss found at Epoch:{}, loss:{}'.format(epoch, best_loss))
                output = f'./fasterrcnn_model_{validation_index}.pt'
                torch.save(model.state_dict(), f'./fasterrcnn_model_{validation_index}.pt')
        
        
        print("Fold is {}, avg loss:{} best loss{} ".format(validation_index, avg_loss,best_loss))
                    
#for fold in range(num_folds):
run(0)   
    

In [ ]:
model.load_state_dict(torch.load("./fasterrcnn_model_0.pt"))
model.to(device)
model.eval()

In [ ]:
test = pd.read_csv("../input/vinbigdata-256-image-dataset/vinbigdata/test.csv")
train = pd.read_csv("../input/vinbigdata-256-image-dataset/vinbigdata/train.csv")

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

test_dataset = VinBigData(df, config["image_dir"],istransforms=True ,transforms=get_transform(train=True))

test_data_loader = DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=False,
    num_workers=4,
    drop_last=False,
    collate_fn=collate_fn
)


In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

test_dataset = VinBigData(test, config["test_dir"], istransforms=True, transforms=get_transform(train=False), isTest=True)

test_data_loader = DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=False,
    num_workers=4,
    drop_last=False,
    collate_fn=collate_fn
)

In [ ]:
def format_prediction_string(labels, boxes, scores):
    pred_strings = []
    for j in zip(labels, scores, boxes):
        pred_strings.append("{0} {1:.4f} {2} {3} {4} {5}".format(
            j[0], j[1], j[2][0], j[2][1], j[2][2], j[2][3]))

    return " ".join(pred_strings)

In [ ]:
threshold = 0.5
results = []

with torch.no_grad():
    
    for images, image_ids in test_data_loader:
        
        #print(images[0].shape)
        images = list(image.to(device) for image in images)
        outputs = model(images)
        
        for i, image in enumerate(images):
            
            image_id = image_ids[i]
            
            result = {
                'image_id': image_id,
                'PredictionString': '14 1.0 0 0 1 1'
            }
        
            print(outputs[i]['scores'])
            boxes = outputs[i]['boxes'].data.cpu().numpy()
            labels = outputs[i]['labels'].data.cpu().numpy()
            scores = outputs[i]['scores'].data.cpu().numpy()
            
            if len(boxes) > 0:
                

                labels = labels - 1
                labels[labels == -1] = 14
                
                selected = scores >= threshold

                boxes = boxes[selected].astype(np.float32)
                scores = scores[selected]
                labels = labels[selected]
                
                print(labels)
                
                if len(boxes) > 0:
                    result = {
                        'image_id':image_id,
                        'PredictionString': format_prediction_string(labels, boxes, scores)
                    }
                    
                    
            results.append(result)

In [ ]:
outputs[0]['boxes'].data.cpu().numpy().shape

In [ ]:
outputs[0]['labels'].data.cpu().numpy().shape

In [ ]:
test_df = pd.DataFrame(results, columns=['image_id', 'PredictionString'])
test_df.head(20)

In [ ]:
test_df.to_csv('./submission.csv', index=False)

In [ ]:
numofRows = 3
grid = ["aabba","aabbba","aaacb"]

visited = [len(grid[0]) * [0]] * len(grid)

In [ ]:
visited

In [ ]:
queue = [grid[0][0]]

moves = [[1,0],[-1,0],[0,1],[0,-1]]
row_max = numofRows
col_max = len(grid[0])

start_row = start_col = 0

while len(queue):
    
    
    prev = grid[start_row][start_col]
    for i in range(len(moves)):
    
        if (start_row >= row_max or start_col >= col_max or grid[start_row][start_col] != prev):
        
        
    
    
    

In [ ]:
lis = [["item1",10,15], ["item2",3,4], ["item3",17,8],["item4",27,3]]

sortParamter= 2
sortOrder = 0
itemsPerPage = 2
pageNumber = 1


lis.sort(reverse = sortOrder, key = lambda x:x[sortParamter])

In [ ]:
lis[pageNumber*itemsPerPage : pageNumber*itemsPerPage+1+itemsPerPage]